# Importaciones

In [1]:
import pandas as pd
import xlrd

# ETL

## Carga de datos y union

In [2]:
# Leer los archivos Excel y añadir una columna 'fecha' con la fecha correspondiente al primer día del mes
data_enero = pd.read_excel(r'Datalake\enero.xls')
data_enero['fecha'] = pd.to_datetime('2023-01-01')

data_febrero = pd.read_excel(r'Datalake\febrero.xls')
data_febrero['fecha'] = pd.to_datetime('2023-02-01')

data_marzo = pd.read_excel(r'Datalake\marzo.xls')
data_marzo['fecha'] = pd.to_datetime('2023-03-01')

data_abril = pd.read_excel(r'Datalake\abril.xls')
data_abril['fecha'] = pd.to_datetime('2023-04-01')

data_mayo = pd.read_excel(r'Datalake\mayo.xls')
data_mayo['fecha'] = pd.to_datetime('2023-05-01')


WARNING *** file size (31123) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** file size (33339) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** file size (32080) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** file size (30924) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
WARNING *** file size (30235) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPA

In [3]:
# Combinar todos los dataframes en uno solo
df_data = pd.concat([data_enero, data_febrero, data_marzo, data_abril, data_mayo], ignore_index=True)

In [4]:
df_data = df_data[['codigo', 'nombre', 'unidades', 'venta', 'fecha']]

# Convertir la columna 'fecha' a tipo datetime y crear una columna 'mes'
df_data['fecha'] = pd.to_datetime(df_data['fecha'])
df_data['mes'] = df_data['fecha'].dt.month_name()

In [5]:
print(df_data.info())
df_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   codigo    771 non-null    int64         
 1   nombre    771 non-null    object        
 2   unidades  771 non-null    float64       
 3   venta     771 non-null    float64       
 4   fecha     771 non-null    datetime64[ns]
 5   mes       771 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 36.3+ KB
None


,codigo,nombre,unidades,venta,fecha,mes
0,350,Mozzarella Gr.,276.0,1832032.51,2023-01-01,January
1,351,Fugazzeta Gr.,114.0,824752.44,2023-01-01,January
2,352,Fugazza Gr.,3.0,21500.00,2023-01-01,January
3,353,Jamon Gr.,79.0,571000.00,2023-01-01,January
4,354,Napolitana Gr.,181.5,1308785.44,2023-01-01,January


## Correciones de nombres

In [6]:
# Corregir mayúsculas y minúsculas
df_data['nombre'] = df_data['nombre'].str.capitalize()

# Usar replace() para eliminar los puntos
df_data['nombre'] = df_data['nombre'].str.replace('.', '')

In [7]:
df_data['nombre'] = df_data['nombre'].str.replace('Pesto brie cherry ch', 'Pesto cherry brie ch')
df_data['nombre'] = df_data['nombre'].str.replace('Salchi a caballo mitad', 'Salchichas a caballo ch')

In [8]:
df_data = df_data[df_data['nombre'] != 'A caballo']
df_data = df_data[df_data['nombre'] != 'Piza mitad y mitad']
df_data = df_data[df_data['nombre'] != 'Promo 5 precocidas']

## Pizzas "Normales" y "Especiales"

In [9]:
# Lista de nombres de interés
nombres_interes = [
    'Degustacion de pizzetas', 'Tacc especial', 'Tacc fugazzeta', 'Tacc hace tu pizza', 'Tacc jamon',
    'Tacc marga napo', 'Tacc margarita', 'Tacc muzzarella', 'Tacc napo jamon', 'Tacc napolitana',
    'Tacc rucula', 'Vegana hace tu pizza', 'Vegana pizza especial', 'Vegana pizza muzza'
]

# Filtrar el DataFrame por los nombres de interés
df_especiales = df_data[df_data['nombre'].isin(nombres_interes)]

df_normales = df_data[~df_data['nombre'].isin(nombres_interes)]

In [10]:
def combinar_registros(df):
    # Extraer el nombre base de la pizza
    df['nombre_pizza'] = df['nombre'].str[:-3]  # Eliminar los últimos 3 caracteres (incluyendo el espacio antes de 'gr' o 'ch')
    
    # Crear listas vacías para almacenar los datos combinados
    registros_combinados = []

    # Iterar sobre cada nombre base de pizza único
    for nombre_pizza in df['nombre_pizza'].unique():
        # Filtrar registros para el nombre base de la pizza actual
        df_pizza = df[df['nombre_pizza'] == nombre_pizza]
        
        # Iterar sobre cada mes único
        for mes in df_pizza['mes'].unique():
            # Filtrar registros para el nombre base de la pizza y mes actual
            df_pizza_mes = df_pizza[df_pizza['mes'] == mes]
            
            # Encontrar las unidades y ventas de 'gr' y 'ch' para el mes actual
            unidades_gr = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('gr')]['unidades'].sum()
            unidades_ch = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('ch')]['unidades'].sum()
            venta_gr = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('gr')]['venta'].sum()
            venta_ch = df_pizza_mes[df_pizza_mes['nombre'].str.endswith('ch')]['venta'].sum()

            # Calcular las unidades y ventas totales
            unidades_total = unidades_gr + unidades_ch
            venta_total = venta_gr + venta_ch
            
            # Tomar solo un registro como base, ya que deberían ser iguales para la misma pizza y mes
            base_registro = df_pizza_mes.iloc[0]
            
            # Crear un diccionario con la información combinada por mes
            registro_combinado = {
                'codigo': base_registro['codigo'],
                'nombre': nombre_pizza,
                'fecha': base_registro['fecha'],
                'mes': mes,
                'unidades_gr': unidades_gr,
                'unidades_ch': unidades_ch,
                'venta_gr': venta_gr,
                'venta_ch': venta_ch,
                'unidades_total': unidades_total,
                'venta_total': venta_total
            }
            
            # Agregar el registro combinado a la lista
            registros_combinados.append(registro_combinado)
    
    # Convertir la lista de registros combinados en un DataFrame
    df_combinado = pd.DataFrame(registros_combinados)
    
    return df_combinado


In [11]:
# Ejemplo de uso de la función
df_normales = combinar_registros(df_normales)

C:\Users\Luka\AppData\Local\Temp\ipykernel_6752\2128083658.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nombre_pizza'] = df['nombre'].str[:-3]  # Eliminar los últimos 3 caracteres (incluyendo el espacio antes de 'gr' o 'ch')


## Carga al Datawarehouse

In [12]:
df_normales.to_csv(r'Datawarehouse\pizzas_normales.csv', index=False)
df_especiales.to_csv(r'Datawarehouse\pizzas_especiales.csv', index=False)